In [1]:
# %% [markdown]
# # 逆向季度轮动策略回测
# 
# ## 导入模块

# %%
import backtrader as bt
import backtrader.analyzers as btanalyzers
import pandas as pd
import yfinance as yf
import akshare as ak
from datetime import datetime, timedelta
import os
import numpy as np
import warnings
import time

warnings.filterwarnings('ignore')

In [2]:
# %% [markdown]
# ## 全局变量设置

# %%
# 资产配置字典
INTL_ETFS = {
    'ETH-USD': 'ETH',
    'QQQ': '纳斯达克ETF',
    'SPY': '标普500ETF'
}

CHINA_ETFS = {
    '159980': '有色金属ETF',
    '513010': '恒生科技',
    '159892': '恒生医药',
    '159934': '黄金ETF',
    '159985': '豆粕ETF',
    '510880': '红利ETF',
    '516780': '稀土ETF'
}

# 资产名称映射
ASSET_NAME_MAP = {
    '159980': '有色金属ETF',
    '513010': '恒生科技ETF',
    '159892': '恒生医药ETF',
    '159934': '黄金ETF',
    '159985': '豆粕ETF',
    '510880': '红利ETF', #563020 #510880
    '516780': '稀土ETF',
    'ETH-USD': 'ETH',
    'QQQ': '纳斯达克ETF',
    'SPY': '标普500ETF'
}

# 回测参数
START_DATE = '2021-07-01'
END_DATE = '2025-08-10'
INITIAL_CASH = 10000
COMMISSION = 0.001  # 0.1%交易佣金

In [3]:
# %% [markdown]
# ## 数据下载函数

# %%
def download_intl_etf(ticker, start_date, end_date):
    """使用yfinance下载国际ETF数据"""
    print(f"正在下载国际ETF {ticker} 数据 ({start_date} 至 {end_date})...")
    try:
        # 增加一周缓冲期
        adjusted_start = (datetime.strptime(start_date, '%Y-%m-%d') - timedelta(days=7)).strftime('%Y-%m-%d')
        data = yf.download(ticker, start=adjusted_start, end=end_date, progress=False)
        
        if data.empty:
            print(f"{ticker} 下载数据为空")
            return None
            
        # 规范列名
        data = data.rename(columns={
            'Open': 'open',
            'High': 'high',
            'Low': 'low',
            'Close': 'close',
            'Volume': 'volume'
        })
        
        # 确保所有必需列存在
        required_cols = ['open', 'high', 'low', 'close']
        if not all(col in data.columns for col in required_cols):
            print(f"{ticker} 缺少必要列")
            return None
            
        # 添加volume列如果不存在
        if 'volume' not in data.columns:
            data['volume'] = 0
            
        # 选择并重排列
        data = data[['open', 'high', 'low', 'close', 'volume']]
        
        # 重置索引并填充缺失值
        data = data.reset_index()
        data.fillna(method='ffill', inplace=True)
        data = data.dropna()
        data['Date'] = pd.to_datetime(data['Date'])
        data.set_index('Date', inplace=True)
        
        # 截取指定时间范围
        start_dt = pd.to_datetime(start_date)
        end_dt = pd.to_datetime(end_date)
        data = data.loc[start_dt:end_dt]
        
        print(f"{ticker} 数据下载成功，{len(data)} 条记录，时间范围: {data.index[0].date()} 至 {data.index[-1].date()}")
        return data
    except Exception as e:
        print(f"下载国际ETF {ticker} 失败: {str(e)}")
        return None

def download_china_etf(ticker, start_date, end_date, retry=3):
    """使用akshare下载中国ETF数据，带重试机制"""
    print(f"正在下载中国ETF {ticker} 数据 ({start_date} 至 {end_date})...")
    for attempt in range(retry):
        try:
            # 增加一周缓冲期
            adjusted_start = (datetime.strptime(start_date, '%Y-%m-%d') - timedelta(days=7)).strftime('%Y%m%d')
            adjusted_end = (datetime.strptime(end_date, '%Y-%m-%d') + timedelta(days=7)).strftime('%Y%m%d')
            
            # 使用akshare的fund_etf_hist_em接口
            df = ak.fund_etf_hist_em(symbol=ticker, period="daily", 
                                    start_date=adjusted_start, 
                                    end_date=adjusted_end)
            
            if df.empty:
                print(f"{ticker} 下载数据为空")
                return None
                
            # 规范列名
            df = df.rename(columns={
                '日期': 'date',
                '开盘': 'open',
                '收盘': 'close',
                '最高': 'high',
                '最低': 'low',
                '成交量': 'volume'
            })
            
            # 转换日期格式
            df['date'] = pd.to_datetime(df['date'])
            
            # 设置日期索引
            df.set_index('date', inplace=True)
            df.index.name = 'Date'
            
            # 截取指定时间范围
            start_dt = pd.to_datetime(start_date)
            end_dt = pd.to_datetime(end_date)
            df = df.loc[start_dt:end_dt]
            
            if df.empty:
                print(f"{ticker} 在指定时间范围内无数据")
                return None
                
            # 确保数据类型正确
            for col in ['open', 'high', 'low', 'close']:
                df[col] = pd.to_numeric(df[col], errors='coerce')
            df['volume'] = pd.to_numeric(df['volume'], errors='coerce')
            
            # 处理可能的缺失值
            df.dropna(subset=['open', 'high', 'low', 'close'], inplace=True)
            
            # 前向填充缺失值
            df.fillna(method='ffill', inplace=True)
            
            # 选择并重排列
            df = df[['open', 'high', 'low', 'close', 'volume']]
            
            print(f"{ticker} 数据下载成功，{len(df)} 条记录，时间范围: {df.index[0].date()} 至 {df.index[-1].date()}")
            return df
        except Exception as e:
            if attempt < retry - 1:
                wait_time = (attempt + 1) * 5
                print(f"下载 {ticker} 失败，{wait_time}秒后重试... ({str(e)})")
                time.sleep(wait_time)
            else:
                print(f"下载中国ETF {ticker} 失败: {str(e)}")
                return None

In [4]:
# %% [markdown]
# ## 策略类定义

# %%
class QuarterlyContrarianRotation(bt.Strategy):
    params = (
        ('selection_count', 4),          # 每次选择的资产数量
        ('rebalance_months', [1, 4, 7, 10]),  # 调仓月份 (1月、4月、7月、10月)
        ('quarter_trading_days', 63),    # 大约一个季度的交易日数
    )

    def __init__(self):
        self.assets = self.datas
        self.asset_names = [d._name for d in self.datas]
        print(f"策略初始化完成，可交易资产: {[ASSET_NAME_MAP.get(name, name) for name in self.asset_names]}")
        
        # 初始化数据结构
        self.trade_log = []
        self.quarterly_returns = {}
        self.current_year = None
        self.year_start_value = None
        self.annual_returns = {}
        self.last_rebalance = None

    def next(self):
        dt = self.datas[0].datetime.date(0)
        
        # 初始化年度回报
        if self.current_year is None:
            self.current_year = dt.year
            self.year_start_value = self.broker.getvalue()
            self.annual_returns[self.current_year] = 0
        
        # 年度更替
        if dt.year != self.current_year:
            self.current_year = dt.year
            self.year_start_value = self.broker.getvalue()
            self.annual_returns[self.current_year] = 0
        
        # 更新年度回报
        self.annual_returns[self.current_year] = (self.broker.getvalue() / self.year_start_value - 1) * 100
        
        # 检查是否是调仓月份
        if dt.month in self.params.rebalance_months:
            # 确保有足够的历史数据
            if len(self.data0) < self.params.quarter_trading_days + 5:
                return
          
            # 检查是否是该月第一次出现
            if self.last_rebalance is None or self.last_rebalance.month != dt.month:
                self.rebalance_portfolio(dt)
                self.last_rebalance = dt

    def rebalance_portfolio(self, dt):
        print(f"\n=== {dt} 季度调仓 ===")
        
        # 计算上个季度的收益率（使用固定交易日间隔）
        quarter_returns = {}
        lookback = self.params.quarter_trading_days
        
        for data in self.assets:
            asset_name = data._name
            display_name = ASSET_NAME_MAP.get(asset_name, asset_name)
            
            # 确保有足够的历史数据
            if len(data) < lookback + 1:
                continue
                
            try:
                # 获取价格 - 使用固定交易日间隔
                start_price = data.close[-lookback]
                end_price = data.close[0]
                
                if start_price > 0 and end_price > 0:
                    return_pct = (end_price - start_price) / start_price * 100
                    quarter_returns[asset_name] = return_pct
                    # 获取实际日期用于调试
                    start_date = data.datetime.date(-lookback)
                    print(f"{display_name}: {return_pct:.2f}% (日期: {start_date} -> {dt})")
                else:
                    print(f"{display_name} 价格无效: start={start_price}, end={end_price}")
            except Exception as e:
                print(f"{display_name} 数据访问错误: {str(e)}")
        
        if len(quarter_returns) < self.params.selection_count:
            print(f"{dt} 有效资产不足({len(quarter_returns)}), 需要{self.params.selection_count}个, 跳过调仓")
            return
        
        # 按收益率升序排序，选择表现最差的4个
        sorted_assets = sorted(quarter_returns.items(), key=lambda x: x[1])
        selected_assets = [x[0] for x in sorted_assets[:self.params.selection_count]]
        selected_names = [ASSET_NAME_MAP.get(a, a) for a in selected_assets]
        print(f"选中资产: {selected_names}")
        
        self.quarterly_returns[dt] = quarter_returns
        
        # 获取当前持仓
        current_positions = {}
        for data in self.assets:
            pos = self.getposition(data).size
            if pos > 0:
                current_positions[data._name] = pos
        
        total_value = self.broker.getvalue()
        target_per_asset = total_value * 0.25
        
        print(f"总资产: {total_value:.2f}, 目标每资产: {target_per_asset:.2f}")
        
        # 调整仓位
        for data in self.assets:
            asset_name = data._name
            display_name = ASSET_NAME_MAP.get(asset_name, asset_name)
            current_pos = self.getposition(data).size
            current_value = current_pos * data.close[0] if current_pos > 0 else 0
            
            if asset_name in selected_assets:
                if asset_name in current_positions:
                    # 调整仓位
                    diff = target_per_asset - current_value
                    if abs(diff) > 1:
                        if diff > 0:
                            size = diff / data.close[0]
                            self.buy(data, size=size)
                            self.log_trade(dt, asset_name, display_name, 'BUY', size, data.close[0])
                            print(f"增持 {display_name}: {size:.2f} 股")
                        else:
                            size = abs(diff) / data.close[0]
                            self.sell(data, size=size)
                            self.log_trade(dt, asset_name, display_name, 'SELL', size, data.close[0])
                            print(f"减持 {display_name}: {size:.2f} 股")
                else:
                    # 新买入
                    size = target_per_asset / data.close[0]
                    self.buy(data, size=size)
                    self.log_trade(dt, asset_name, display_name, 'BUY', size, data.close[0])
                    print(f"买入 {display_name}: {size:.2f} 股")
            else:
                if asset_name in current_positions:
                    # 清仓
                    self.close(data)
                    self.log_trade(dt, asset_name, display_name, 'CLOSE', current_pos, data.close[0])
                    print(f"清仓 {display_name}: {current_pos:.2f} 股")
    
    def log_trade(self, date, asset, display_name, action, size, price):
        self.trade_log.append({
            'Date': date,
            'Asset': display_name,
            'Action': action,
            'Size': size,
            'Price': price,
            'Value': size * price
        })
    
    def stop(self):
        # 在结束时处理持仓
        dt = self.datas[0].datetime.date(0)
        print(f"\n结束日期 {dt} 处理持仓")
        
        # 清空所有持仓
        for data in self.assets:
            pos = self.getposition(data).size
            if pos > 0:
                try:
                    display_name = ASSET_NAME_MAP.get(data._name, data._name)
                    self.close(data)
                    self.log_trade(dt, data._name, display_name, 'CLOSE', pos, data.close[0])
                    print(f"清仓 {display_name}: {pos:.2f} 股")
                except Exception as e:
                    print(f"清仓 {data._name} 失败: {str(e)}")
        
        # 创建数据框
        self.trade_df = pd.DataFrame(self.trade_log)
        
        # 计算年度回报
        years = sorted(set(int(y) for y in self.annual_returns.keys()))
        annual_returns = [self.annual_returns.get(y, 0) for y in years]
        self.annual_return_df = pd.DataFrame({
            'Year': years,
            'Return(%)': annual_returns
        })
        
        # 季度回报
        quarter_returns = []
        for date, returns in self.quarterly_returns.items():
            row = {'Date': date}
            # 使用显示名称
            for asset, ret in returns.items():
                display_name = ASSET_NAME_MAP.get(asset, asset)
                row[display_name] = ret
            quarter_returns.append(row)
        self.quarterly_return_df = pd.DataFrame(quarter_returns)

In [5]:
class QuarterlyContrarianRotationWithDip2(bt.Strategy):
    params = (
        ('selection_count', 4),          # 每次选择的资产数量
        ('rebalance_months', [1, 4, 7, 10]),  # 调仓月份 (1月、4月、7月、10月)
        ('quarter_trading_days', 63),    # 大约一个季度的交易日数
        ('dip_threshold', 0.04),         # 波谷确认阈值（价格下跌超过5%）
        ('dip_confirmation_period', 2),  # 波谷确认周期（连续上涨天数）
        ('min_data_length', 60),        # 资产所需的最小数据长度
        ('observation_period', 22),      # 观察期长度（交易日）
        ('max_drawdown_threshold', -0.40)  # 新增：最大回撤阈值（-8%）
    )

    def __init__(self):
        self.assets = self.datas
        self.asset_names = [d._name for d in self.datas]
        print(f"策略初始化完成，可交易资产: {[ASSET_NAME_MAP.get(name, name) for name in self.asset_names]}")
        
        # 初始化数据结构
        self.trade_log = []
        self.quarterly_returns = {}
        self.current_year = None
        self.year_start_value = None
        self.annual_returns = {}
        self.last_rebalance = None
        
        # 波谷检测相关变量
        self.observation_start_date = None  # 观察期开始日期
        self.observation_start_index = None # 观察期开始时的数据索引
        self.selected_assets = []           # 本季度选中的资产
        self.asset_low_points = {}          # 资产的最低点记录
        self.asset_dip_confirmed = {}       # 资产波谷确认状态
        self.asset_force_bought = {}        # 资产是否已被强制买入
        
        # 为每个资产记录观察期开始时的价格
        self.observation_start_prices = {}
        
        # 为每个资产添加指标
        for data in self.assets:
            asset_name = data._name
            
            # 记录每个资产的最低点
            self.asset_low_points[asset_name] = {
                'price': float('inf'),
                'date': None
            }
            
            # 初始化波谷确认状态
            self.asset_dip_confirmed[asset_name] = False
            self.asset_force_bought[asset_name] = False
            
            # 检查资产数据是否有效
            if len(data) < self.params.min_data_length:
                print(f"警告: {ASSET_NAME_MAP.get(asset_name, asset_name)} 数据不足 ({len(data)} 条记录), "
                      f"需要至少 {self.params.min_data_length} 条记录")

    def next(self):
        dt = self.datas[0].datetime.date(0)
        current_index = len(self.data0)  # 当前数据索引
        
        # 初始化年度回报
        if self.current_year is None:
            self.current_year = dt.year
            self.year_start_value = self.broker.getvalue()
            self.annual_returns[self.current_year] = 0
        
        # 年度更替
        if dt.year != self.current_year:
            self.current_year = dt.year
            self.year_start_value = self.broker.getvalue()
            self.annual_returns[self.current_year] = 0
        
        # 更新年度回报 - 添加严格检查
        if self.year_start_value and self.year_start_value > 0:
            self.annual_returns[self.current_year] = (self.broker.getvalue() / self.year_start_value - 1) * 100
        else:
            self.annual_returns[self.current_year] = 0
        
        # 检查是否是调仓月份
        if dt.month in self.params.rebalance_months:
            # 确保有足够的历史数据
            if len(self.data0) < self.params.quarter_trading_days + 5:
                return
                
            # 检查是否是该月第一次出现
            if self.last_rebalance is None or self.last_rebalance.month != dt.month:
                # 重置观察期和波谷检测状态
                self.observation_start_date = dt
                self.observation_start_index = current_index  # 记录当前索引
                self.selected_assets = self.select_assets_for_quarter(dt)
                
                # 重置资产状态
                for asset in self.asset_names:
                    self.asset_dip_confirmed[asset] = False
                    self.asset_force_bought[asset] = False
                
                # 记录观察期开始时的价格
                self.observation_start_prices = {}
                for data in self.assets:
                    # 只记录选中的资产
                    if data._name in self.selected_assets:
                        # 确保有数据且价格有效
                        if len(data) > 0 and data.close[0] > 0:
                            self.observation_start_prices[data._name] = data.close[0]
                        else:
                            print(f"无法记录 {data._name} 的观察期起始价格: 数据长度={len(data)}, 价格={data.close[0] if len(data) > 0 else 'N/A'}")
                
                # 清仓非选中资产（仅在此处执行一次）
                self.close_non_selected_assets(dt)
                
                self.reset_dip_detection()
                self.last_rebalance = dt
                
                if self.selected_assets:
                    print(f"\n=== {dt} 季度观察期开始，选中的资产: {[ASSET_NAME_MAP.get(a, a) for a in self.selected_assets]} ===")
                    print(f"观察期将持续 {self.params.observation_period} 个交易日")
                else:
                    print(f"\n=== {dt} 季度观察期开始，但未选中任何资产 ===")
        
        # 如果处于观察期且有选中的资产，检测波谷
        if self.observation_start_date and self.selected_assets:
            self.detect_dips(dt)
            
            # 检查观察期是否结束（22个交易日后）
            days_elapsed = current_index - self.observation_start_index
            if days_elapsed >= self.params.observation_period:
                print(f"\n=== 观察期结束 ({days_elapsed}个交易日) ===")
                self.force_buy_at_end_of_observation(dt)
                # 重置观察期
                self.observation_start_date = None
        
        # 执行买入操作（当波谷被确认时）
        self.execute_dip_buys(dt)

    def close_non_selected_assets(self, dt):
        """清仓所有非选中的资产（仅在季度开始时调用）"""
        # 获取当前持仓
        current_positions = {}
        for data in self.assets:
            if len(data) < 1:  # 确保有数据
                continue
            pos = self.getposition(data).size
            if pos > 0:
                current_positions[data._name] = pos
        
        # 清仓非选中资产
        for data in self.assets:
            asset_name = data._name
            
            # 只处理非选中资产
            if asset_name not in self.selected_assets:
                if asset_name in current_positions:
                    display_name = ASSET_NAME_MAP.get(asset_name, asset_name)
                    pos = current_positions[asset_name]
                    
                    # 确保价格有效
                    if data.close[0] > 0:
                        self.close(data)
                        self.log_trade(dt, asset_name, display_name, 'CLOSE', pos, data.close[0])
                        print(f"清仓非选中资产 {display_name}: {pos:.2f} 股")
                    else:
                        print(f"无法清仓 {display_name}: 价格无效 ({data.close[0]})")

    def select_assets_for_quarter(self, dt):
        """选择本季度表现最差的资产，但跳过回撤超过阈值的基金"""
        print(f"\n=== {dt} 季度资产选择 (跳过回撤 > {self.params.max_drawdown_threshold*100:.1f}%的基金) ===")
        quarter_returns = {}
        lookback = self.params.quarter_trading_days
        
        for data in self.assets:
            asset_name = data._name
            display_name = ASSET_NAME_MAP.get(asset_name, asset_name)
            
            # 确保有足够的历史数据
            if len(data) < lookback + 1:
                print(f"{display_name} 数据不足 ({len(data)} 条记录), 需要至少 {lookback+1} 条记录, 跳过")
                continue
                
            try:
                # 获取价格 - 添加严格检查
                if len(data.close) < lookback + 1:
                    print(f"{display_name} 数据不足，无法计算回报")
                    continue
                    
                start_price = data.close[-lookback]
                end_price = data.close[0]
                
                # 确保价格有效且不为零
                if start_price > 0 and end_price > 0:
                    return_pct = (end_price - start_price) / start_price * 100
                    
                    # 检查回撤是否超过阈值
                    if return_pct < self.params.max_drawdown_threshold * 100:
                        print(f"跳过 {display_name}: 上季度回撤 {return_pct:.2f}% 超过阈值 {self.params.max_drawdown_threshold*100:.1f}%")
                        continue
                    
                    quarter_returns[asset_name] = return_pct
                    # 获取实际日期用于调试
                    start_date = data.datetime.date(-lookback)
                    print(f"{display_name}: {return_pct:.2f}% (日期: {start_date} -> {dt})")
                else:
                    print(f"{display_name} 价格无效或为零: start={start_price}, end={end_price}")
                    # 如果价格无效，跳过该资产
                    continue
            except Exception as e:
                print(f"{display_name} 数据访问错误: {str(e)}")
                continue
        
        if len(quarter_returns) < self.params.selection_count:
            print(f"{dt} 有效资产不足({len(quarter_returns)}), 需要{self.params.selection_count}个, 跳过调仓")
            return []
        
        # 按收益率升序排序，选择表现最差的4个
        sorted_assets = sorted(quarter_returns.items(), key=lambda x: x[1])
        selected_assets = [x[0] for x in sorted_assets[:self.params.selection_count]]
        selected_names = [ASSET_NAME_MAP.get(a, a) for a in selected_assets]
        print(f"选中资产: {selected_names}")
        
        self.quarterly_returns[dt] = quarter_returns
        return selected_assets

    def reset_dip_detection(self):
        """重置波谷检测状态"""
        for asset in self.asset_low_points:
            self.asset_low_points[asset] = {
                'price': float('inf'),
                'date': None
            }
            self.asset_dip_confirmed[asset] = False

    def detect_dips(self, dt):
        """检测资产的波谷点"""
        for data in self.assets:
            asset_name = data._name
            display_name = ASSET_NAME_MAP.get(asset_name, asset_name)
            
            # 只关注选中的资产
            if asset_name not in self.selected_assets:
                continue
                
            # 如果已经确认波谷，跳过
            if self.asset_dip_confirmed[asset_name]:
                continue
                
            # 确保有足够的数据
            if len(data) < 1:
                continue
                
            current_price = data.close[0]
            
            # 更新最低点
            if current_price < self.asset_low_points[asset_name]['price']:
                self.asset_low_points[asset_name]['price'] = current_price
                self.asset_low_points[asset_name]['date'] = dt
            
            # 检查是否出现波谷确认信号
            if self.is_dip_confirmed(data, asset_name):
                self.asset_dip_confirmed[asset_name] = True
                print(f"{display_name} 在 {dt} 确认波谷，价格为 {current_price:.4f}")

    def is_dip_confirmed(self, data, asset_name):
        """确认是否出现波谷（连续上涨确认反弹）"""
        # 检查最近几天是否连续上涨
        if len(data) < self.params.dip_confirmation_period + 1:
            return False
            
        # 确保有足够的数据计算连续上涨
        if len(data.close) < self.params.dip_confirmation_period + 1:
            return False
            
        for i in range(self.params.dip_confirmation_period):
            # 使用索引访问历史数据
            if data.close[-i] <= data.close[-i-1]:
                return False
        
        # 检查观察期开始价格是否存在
        if asset_name not in self.observation_start_prices:
            return False
            
        start_price = self.observation_start_prices[asset_name]
        current_price = data.close[0]
        
        # 确保起始价格有效
        if start_price <= 0:
            return False
            
        # 检查是否下跌超过阈值
        price_drop = (start_price - current_price) / start_price
        if price_drop < self.params.dip_threshold:
            return False
            
        return True

    def execute_dip_buys(self, dt):
        """执行波谷买入操作"""
        # 如果没有选中的资产，跳过
        if not self.selected_assets:
            return
            
        # 获取当前持仓
        current_positions = {}
        for data in self.assets:
            if len(data) < 1:  # 确保有数据
                continue
            pos = self.getposition(data).size
            if pos > 0:
                current_positions[data._name] = pos
        
        total_value = self.broker.getvalue()
        
        # 确保总价值有效
        if total_value <= 0:
            print(f"总资产无效: {total_value:.2f}, 跳过买入操作")
            return
            
        target_per_asset = total_value * 0.25
        
        # 调整仓位 - 只处理选中资产
        for data in self.assets:
            if len(data) < 1:  # 确保有数据
                continue
                
            asset_name = data._name
            display_name = ASSET_NAME_MAP.get(asset_name, asset_name)
            
            # 只处理选中的资产
            if asset_name not in self.selected_assets:
                continue  # 不再在此处清仓
                
            current_pos = self.getposition(data).size
            current_value = current_pos * data.close[0] if current_pos > 0 else 0
            
            # 确保价格有效
            if data.close[0] <= 0:
                print(f"{display_name} 当前价格无效: {data.close[0]}, 跳过买入")
                continue
                
            # 如果波谷已确认且尚未买入
            if self.asset_dip_confirmed[asset_name] and current_pos == 0:
                # 新买入 - 添加检查防止除以零
                if data.close[0] > 0:  # 确保价格有效
                    size = target_per_asset / data.close[0]
                    self.buy(data, size=size)
                    self.log_trade(dt, asset_name, display_name, 'BUY', size, data.close[0])
                    print(f"波谷买入 {display_name}: {size:.2f} 股 @ {data.close[0]:.4f}")
                else:
                    print(f"{display_name} 价格无效: {data.close[0]}, 无法买入")
    
    def force_buy_at_end_of_observation(self, dt):
        """在观察期结束时强制买入尚未购买的资产"""
        print(f"观察期结束，强制买入尚未购买的资产")
        
        # 获取当前持仓
        current_positions = {}
        for data in self.assets:
            if len(data) < 1:  # 确保有数据
                continue
            pos = self.getposition(data).size
            if pos > 0:
                current_positions[data._name] = pos
        
        total_value = self.broker.getvalue()
        
        # 确保总价值有效
        if total_value <= 0:
            print(f"总资产无效: {total_value:.2f}, 跳过强制买入")
            return
            
        target_per_asset = total_value * 0.25
        
        # 处理每个选中的资产
        for data in self.assets:
            if len(data) < 1:  # 确保有数据
                continue
                
            asset_name = data._name
            display_name = ASSET_NAME_MAP.get(asset_name, asset_name)
            
            # 只处理选中的资产
            if asset_name not in self.selected_assets:
                continue
                
            # 如果已经买入（波谷买入或强制买入），跳过
            if self.asset_dip_confirmed[asset_name] or self.asset_force_bought[asset_name]:
                continue
                
            current_pos = self.getposition(data).size
            
            # 如果尚未持仓
            if current_pos == 0:
                # 确保价格有效
                if data.close[0] <= 0:
                    print(f"{display_name} 当前价格无效: {data.close[0]}, 跳过强制买入")
                    continue
                    
                # 强制买入
                size = target_per_asset / data.close[0]
                self.buy(data, size=size)
                self.asset_force_bought[asset_name] = True
                self.log_trade(dt, asset_name, display_name, 'FORCE_BUY', size, data.close[0])
                print(f"观察期结束强制买入 {display_name}: {size:.2f} 股 @ {data.close[0]:.4f}")
    
    def log_trade(self, date, asset, display_name, action, size, price):
        # 添加价格有效性检查
        if price <= 0:
            print(f"警告: {display_name} 交易价格为 {price}，跳过记录")
            return
            
        # 添加大小有效性检查
        if size <= 0:
            print(f"警告: {display_name} 交易量为 {size}，跳过记录")
            return
            
        self.trade_log.append({
            'Date': date,
            'Asset': display_name,
            'Action': action,
            'Size': size,
            'Price': price,
            'Value': size * price
        })
    
    def stop(self):
        # 在结束时处理持仓
        dt = self.datas[0].datetime.date(0)
        print(f"\n结束日期 {dt} 处理持仓")
        
        # 清空所有持仓
        for data in self.assets:
            if len(data) < 1:  # 确保有数据
                continue
                
            pos = self.getposition(data).size
            if pos > 0:
                try:
                    display_name = ASSET_NAME_MAP.get(data._name, data._name)
                    # 添加价格有效性检查
                    if data.close[0] > 0:
                        self.close(data)
                        self.log_trade(dt, data._name, display_name, 'CLOSE', pos, data.close[0])
                        print(f"清仓 {display_name}: {pos:.2f} 股")
                    else:
                        print(f"{display_name} 价格无效: {data.close[0]}, 无法清仓")
                except Exception as e:
                    print(f"清仓 {data._name} 失败: {str(e)}")
        
        # 创建数据框
        if self.trade_log:
            self.trade_df = pd.DataFrame(self.trade_log)
        else:
            print("无交易记录")
            self.trade_df = pd.DataFrame()
        
        # 计算年度回报
        years = sorted(set(int(y) for y in self.annual_returns.keys()))
        annual_returns = [self.annual_returns.get(y, 0) for y in years]
        self.annual_return_df = pd.DataFrame({
            'Year': years,
            'Return(%)': annual_returns
        })
        
        # 季度回报
        quarter_returns = []
        for date, returns in self.quarterly_returns.items():
            row = {'Date': date}
            # 使用显示名称
            for asset, ret in returns.items():
                display_name = ASSET_NAME_MAP.get(asset, asset)
                row[display_name] = ret
            quarter_returns.append(row)
        self.quarterly_return_df = pd.DataFrame(quarter_returns)

In [9]:
class QuarterlyContrarianRotationWithDip(bt.Strategy):
    params = (
        ('selection_count', 4),          # 每次选择的资产数量
        ('rebalance_months', [1, 4, 7, 10]),  # 调仓月份 (1月、4月、7月、10月)
        ('quarter_trading_days', 63),    # 大约一个季度的交易日数
        ('dip_threshold', 0.04),         # 波谷确认阈值（价格下跌超过4%）
        ('dip_confirmation_period', 2),  # 波谷确认周期（连续上涨天数） - 保留但不再使用
        ('rsi_period', 14),              # RSI计算周期
        ('rsi_oversold', 30),            # RSI超卖阈值
        ('volume_ratio', 1.2),           # 成交量放大比例(120%)
        ('min_data_length', 60),         # 资产所需的最小数据长度
        ('observation_period', 22),      # 观察期长度（交易日）
        ('max_drawdown_threshold', -0.40)  # 最大回撤阈值（-40%）
    )

    def __init__(self):
        self.assets = self.datas
        self.asset_names = [d._name for d in self.datas]
        print(f"策略初始化完成，可交易资产: {[ASSET_NAME_MAP.get(name, name) for name in self.asset_names]}")
        
        # 初始化数据结构
        self.trade_log = []
        self.quarterly_returns = {}
        self.current_year = None
        self.year_start_value = None
        self.annual_returns = {}
        self.last_rebalance = None
        
        # 波谷检测相关变量
        self.observation_start_date = None  # 观察期开始日期
        self.observation_start_index = None # 观察期开始时的数据索引
        self.selected_assets = []           # 本季度选中的资产
        self.asset_low_points = {}          # 资产的最低点记录
        self.asset_dip_confirmed = {}       # 资产波谷确认状态
        self.asset_force_bought = {}        # 资产是否已被强制买入
        
        # 为每个资产记录观察期开始时的价格
        self.observation_start_prices = {}
        
        # 为每个资产添加指标
        self.rsi_indicators = {}  # 存储RSI指标
        self.vol_ma_indicators = {}  # 存储成交量均线指标
        
        for data in self.assets:
            asset_name = data._name
            
            # 记录每个资产的最低点
            self.asset_low_points[asset_name] = {
                'price': float('inf'),
                'date': None
            }
            
            # 初始化波谷确认状态
            self.asset_dip_confirmed[asset_name] = False
            self.asset_force_bought[asset_name] = False
            
            # 添加技术指标
            self.rsi_indicators[asset_name] = bt.indicators.RelativeStrengthIndex(
                data.close, period=self.params.rsi_period)
            
            self.vol_ma_indicators[asset_name] = bt.indicators.SimpleMovingAverage(
                data.volume, period=5)  # 5日成交量均线
            
            # 检查资产数据是否有效
            if len(data) < self.params.min_data_length:
                print(f"警告: {ASSET_NAME_MAP.get(asset_name, asset_name)} 数据不足 ({len(data)} 条记录), "
                      f"需要至少 {self.params.min_data_length} 条记录")

    def next(self):
        dt = self.datas[0].datetime.date(0)
        current_index = len(self.data0)  # 当前数据索引
        
        # 初始化年度回报
        if self.current_year is None:
            self.current_year = dt.year
            self.year_start_value = self.broker.getvalue()
            self.annual_returns[self.current_year] = 0
        
        # 年度更替
        if dt.year != self.current_year:
            self.current_year = dt.year
            self.year_start_value = self.broker.getvalue()
            self.annual_returns[self.current_year] = 0
        
        # 更新年度回报 - 添加严格检查
        if self.year_start_value and self.year_start_value > 0:
            self.annual_returns[self.current_year] = (self.broker.getvalue() / self.year_start_value - 1) * 100
        else:
            self.annual_returns[self.current_year] = 0
        
        # 检查是否是调仓月份
        if dt.month in self.params.rebalance_months:
            # 确保有足够的历史数据
            if len(self.data0) < self.params.quarter_trading_days + 5:
                return
                
            # 检查是否是该月第一次出现
            if self.last_rebalance is None or self.last_rebalance.month != dt.month:
                # 重置观察期和波谷检测状态
                self.observation_start_date = dt
                self.observation_start_index = current_index  # 记录当前索引
                self.selected_assets = self.select_assets_for_quarter(dt)
                
                # 重置资产状态
                for asset in self.asset_names:
                    self.asset_dip_confirmed[asset] = False
                    self.asset_force_bought[asset] = False
                
                # 记录观察期开始时的价格
                self.observation_start_prices = {}
                for data in self.assets:
                    # 只记录选中的资产
                    if data._name in self.selected_assets:
                        # 确保有数据且价格有效
                        if len(data) > 0 and data.close[0] > 0:
                            self.observation_start_prices[data._name] = data.close[0]
                        else:
                            print(f"无法记录 {data._name} 的观察期起始价格: 数据长度={len(data)}, 价格={data.close[0] if len(data) > 0 else 'N/A'}")
                
                # 清仓非选中资产（仅在此处执行一次）
                self.close_non_selected_assets(dt)
                
                self.reset_dip_detection()
                self.last_rebalance = dt
                
                if self.selected_assets:
                    print(f"\n=== {dt} 季度观察期开始，选中的资产: {[ASSET_NAME_MAP.get(a, a) for a in self.selected_assets]} ===")
                    print(f"观察期将持续 {self.params.observation_period} 个交易日")
                else:
                    print(f"\n=== {dt} 季度观察期开始，但未选中任何资产 ===")
        
        # 如果处于观察期且有选中的资产，检测波谷
        if self.observation_start_date and self.selected_assets:
            self.detect_dips(dt)
            
            # 检查观察期是否结束（22个交易日后）
            days_elapsed = current_index - self.observation_start_index
            if days_elapsed >= self.params.observation_period:
                print(f"\n=== 观察期结束 ({days_elapsed}个交易日) ===")
                self.force_buy_at_end_of_observation(dt)
                # 重置观察期
                self.observation_start_date = None
        
        # 执行买入操作（当波谷被确认时）
        self.execute_dip_buys(dt)

    def close_non_selected_assets(self, dt):
        """清仓所有非选中的资产（仅在季度开始时调用）"""
        # 获取当前持仓
        current_positions = {}
        for data in self.assets:
            if len(data) < 1:  # 确保有数据
                continue
            pos = self.getposition(data).size
            if pos > 0:
                current_positions[data._name] = pos
        
        # 清仓非选中资产
        for data in self.assets:
            asset_name = data._name
            
            # 只处理非选中资产
            if asset_name not in self.selected_assets:
                if asset_name in current_positions:
                    display_name = ASSET_NAME_MAP.get(asset_name, asset_name)
                    pos = current_positions[asset_name]
                    
                    # 确保价格有效
                    if data.close[0] > 0:
                        self.close(data)
                        self.log_trade(dt, asset_name, display_name, 'CLOSE', pos, data.close[0])
                        print(f"清仓非选中资产 {display_name}: {pos:.2f} 股")
                    else:
                        print(f"无法清仓 {display_name}: 价格无效 ({data.close[0]})")

    def select_assets_for_quarter(self, dt):
        """选择本季度表现最差的资产，但跳过回撤超过阈值的基金"""
        print(f"\n=== {dt} 季度资产选择 (跳过回撤 > {self.params.max_drawdown_threshold*100:.1f}%的基金) ===")
        quarter_returns = {}
        lookback = self.params.quarter_trading_days
        
        for data in self.assets:
            asset_name = data._name
            display_name = ASSET_NAME_MAP.get(asset_name, asset_name)
            
            # 确保有足够的历史数据
            if len(data) < lookback + 1:
                print(f"{display_name} 数据不足 ({len(data)} 条记录), 需要至少 {lookback+1} 条记录, 跳过")
                continue
                
            try:
                # 获取价格 - 添加严格检查
                if len(data.close) < lookback + 1:
                    print(f"{display_name} 数据不足，无法计算回报")
                    continue
                    
                start_price = data.close[-lookback]
                end_price = data.close[0]
                
                # 确保价格有效且不为零
                if start_price > 0 and end_price > 0:
                    return_pct = (end_price - start_price) / start_price * 100
                    
                    # 检查回撤是否超过阈值
                    if return_pct < self.params.max_drawdown_threshold * 100:
                        print(f"跳过 {display_name}: 上季度回撤 {return_pct:.2f}% 超过阈值 {self.params.max_drawdown_threshold*100:.1f}%")
                        continue
                    
                    quarter_returns[asset_name] = return_pct
                    # 获取实际日期用于调试
                    start_date = data.datetime.date(-lookback)
                    print(f"{display_name}: {return_pct:.2f}% (日期: {start_date} -> {dt})")
                else:
                    print(f"{display_name} 价格无效或为零: start={start_price}, end={end_price}")
                    # 如果价格无效，跳过该资产
                    continue
            except Exception as e:
                print(f"{display_name} 数据访问错误: {str(e)}")
                continue
        
        if len(quarter_returns) < self.params.selection_count:
            print(f"{dt} 有效资产不足({len(quarter_returns)}), 需要{self.params.selection_count}个, 跳过调仓")
            return []
        
        # 按收益率升序排序，选择表现最差的4个
        sorted_assets = sorted(quarter_returns.items(), key=lambda x: x[1])
        selected_assets = [x[0] for x in sorted_assets[:self.params.selection_count]]
        selected_names = [ASSET_NAME_MAP.get(a, a) for a in selected_assets]
        print(f"选中资产: {selected_names}")
        
        self.quarterly_returns[dt] = quarter_returns
        return selected_assets

    def reset_dip_detection(self):
        """重置波谷检测状态"""
        for asset in self.asset_low_points:
            self.asset_low_points[asset] = {
                'price': float('inf'),
                'date': None
            }
            self.asset_dip_confirmed[asset] = False

    def detect_dips(self, dt):
        """检测资产的波谷点"""
        for data in self.assets:
            asset_name = data._name
            display_name = ASSET_NAME_MAP.get(asset_name, asset_name)
            
            # 只关注选中的资产
            if asset_name not in self.selected_assets:
                continue
                
            # 如果已经确认波谷，跳过
            if self.asset_dip_confirmed[asset_name]:
                continue
                
            # 确保有足够的数据
            if len(data) < 1:
                continue
                
            current_price = data.close[0]
            
            # 更新最低点
            if current_price < self.asset_low_points[asset_name]['price']:
                self.asset_low_points[asset_name]['price'] = current_price
                self.asset_low_points[asset_name]['date'] = dt
            
            # 检查是否出现波谷确认信号
            if self.is_dip_confirmed(data, asset_name):
                self.asset_dip_confirmed[asset_name] = True
                print(f"{display_name} 在 {dt} 确认波谷，价格为 {current_price:.4f}")

    def is_dip_confirmed(self, data, asset_name):
        """使用RSI+成交量组合确认波谷"""
        # 检查观察期开始价格是否存在
        if asset_name not in self.observation_start_prices:
            return False
            
        start_price = self.observation_start_prices[asset_name]
        current_price = data.close[0]
        
        # 确保起始价格有效
        if start_price <= 0 or current_price <= 0:
            return False
            
        # 检查是否下跌超过阈值
        price_drop = (start_price - current_price) / start_price
        if price_drop < self.params.dip_threshold:
            return False
            
        # 确保有足够数据计算RSI
        if len(self.rsi_indicators[asset_name]) < 1:
            return False
            
        # 获取当前RSI值
        rsi_value = self.rsi_indicators[asset_name][0]
        
        # 获取当前成交量
        current_volume = data.volume[0]
        
        # 确保有足够数据计算成交量均线
        if len(self.vol_ma_indicators[asset_name]) < 1:
            return False
            
        # 获取5日成交量均线
        vol_ma = self.vol_ma_indicators[asset_name][0]
        
        # 检查指标是否有效（非NaN且大于0）
        if (rsi_value != rsi_value or vol_ma != vol_ma or  # 检查NaN
            rsi_value <= 0 or vol_ma <= 0 or current_volume <= 0):
            return False
        
        # 波谷确认条件：
        # 1. RSI < 超卖阈值（默认30）
        # 2. 当前成交量 > 5日成交量均线的指定比例（默认120%）
        rsi_condition = rsi_value < self.params.rsi_oversold
        volume_condition = current_volume > vol_ma * self.params.volume_ratio
        
        # 返回是否满足条件
        return rsi_condition and volume_condition

    def execute_dip_buys(self, dt):
        """执行波谷买入操作"""
        # 如果没有选中的资产，跳过
        if not self.selected_assets:
            return
            
        # 获取当前持仓
        current_positions = {}
        for data in self.assets:
            if len(data) < 1:  # 确保有数据
                continue
            pos = self.getposition(data).size
            if pos > 0:
                current_positions[data._name] = pos
        
        total_value = self.broker.getvalue()
        
        # 确保总价值有效
        if total_value <= 0:
            print(f"总资产无效: {total_value:.2f}, 跳过买入操作")
            return
            
        target_per_asset = total_value * 0.25
        
        # 调整仓位 - 只处理选中资产
        for data in self.assets:
            if len(data) < 1:  # 确保有数据
                continue
                
            asset_name = data._name
            display_name = ASSET_NAME_MAP.get(asset_name, asset_name)
            
            # 只处理选中的资产
            if asset_name not in self.selected_assets:
                continue  # 不再在此处清仓
                
            current_pos = self.getposition(data).size
            current_value = current_pos * data.close[0] if current_pos > 0 else 0
            
            # 确保价格有效
            if data.close[0] <= 0:
                print(f"{display_name} 当前价格无效: {data.close[0]}, 跳过买入")
                continue
                
            # 如果波谷已确认且尚未买入
            if self.asset_dip_confirmed[asset_name] and current_pos == 0:
                # 新买入 - 添加检查防止除以零
                if data.close[0] > 0:  # 确保价格有效
                    size = target_per_asset / data.close[0]
                    self.buy(data, size=size)
                    self.log_trade(dt, asset_name, display_name, 'BUY', size, data.close[0])
                    print(f"波谷买入 {display_name}: {size:.2f} 股 @ {data.close[0]:.4f}")
                else:
                    print(f"{display_name} 价格无效: {data.close[0]}, 无法买入")
    
    def force_buy_at_end_of_observation(self, dt):
        """在观察期结束时强制买入尚未购买的资产"""
        print(f"观察期结束，强制买入尚未购买的资产")
        
        # 获取当前持仓
        current_positions = {}
        for data in self.assets:
            if len(data) < 1:  # 确保有数据
                continue
            pos = self.getposition(data).size
            if pos > 0:
                current_positions[data._name] = pos
        
        total_value = self.broker.getvalue()
        
        # 确保总价值有效
        if total_value <= 0:
            print(f"总资产无效: {total_value:.2f}, 跳过强制买入")
            return
            
        target_per_asset = total_value * 0.25
        
        # 处理每个选中的资产
        for data in self.assets:
            if len(data) < 1:  # 确保有数据
                continue
                
            asset_name = data._name
            display_name = ASSET_NAME_MAP.get(asset_name, asset_name)
            
            # 只处理选中的资产
            if asset_name not in self.selected_assets:
                continue
                
            # 如果已经买入（波谷买入或强制买入），跳过
            if self.asset_dip_confirmed[asset_name] or self.asset_force_bought[asset_name]:
                continue
                
            current_pos = self.getposition(data).size
            
            # 如果尚未持仓
            if current_pos == 0:
                # 确保价格有效
                if data.close[0] <= 0:
                    print(f"{display_name} 当前价格无效: {data.close[0]}, 跳过强制买入")
                    continue
                    
                # 强制买入
                size = target_per_asset / data.close[0]
                self.buy(data, size=size)
                self.asset_force_bought[asset_name] = True
                self.log_trade(dt, asset_name, display_name, 'FORCE_BUY', size, data.close[0])
                print(f"观察期结束强制买入 {display_name}: {size:.2f} 股 @ {data.close[0]:.4f}")
    
    def log_trade(self, date, asset, display_name, action, size, price):
        # 添加价格有效性检查
        if price <= 0:
            print(f"警告: {display_name} 交易价格为 {price}，跳过记录")
            return
            
        # 添加大小有效性检查
        if size <= 0:
            print(f"警告: {display_name} 交易量为 {size}，跳过记录")
            return
            
        self.trade_log.append({
            'Date': date,
            'Asset': display_name,
            'Action': action,
            'Size': size,
            'Price': price,
            'Value': size * price
        })
    
    def stop(self):
        # 在结束时处理持仓
        dt = self.datas[0].datetime.date(0)
        print(f"\n结束日期 {dt} 处理持仓")
        
        # 清空所有持仓
        for data in self.assets:
            if len(data) < 1:  # 确保有数据
                continue
                
            pos = self.getposition(data).size
            if pos > 0:
                try:
                    display_name = ASSET_NAME_MAP.get(data._name, data._name)
                    # 添加价格有效性检查
                    if data.close[0] > 0:
                        self.close(data)
                        self.log_trade(dt, data._name, display_name, 'CLOSE', pos, data.close[0])
                        print(f"清仓 {display_name}: {pos:.2f} 股")
                    else:
                        print(f"{display_name} 价格无效: {data.close[0]}, 无法清仓")
                except Exception as e:
                    print(f"清仓 {data._name} 失败: {str(e)}")
        
        # 创建数据框
        if self.trade_log:
            self.trade_df = pd.DataFrame(self.trade_log)
        else:
            print("无交易记录")
            self.trade_df = pd.DataFrame()
        
        # 计算年度回报
        years = sorted(set(int(y) for y in self.annual_returns.keys()))
        annual_returns = [self.annual_returns.get(y, 0) for y in years]
        self.annual_return_df = pd.DataFrame({
            'Year': years,
            'Return(%)': annual_returns
        })
        
        # 季度回报
        quarter_returns = []
        for date, returns in self.quarterly_returns.items():
            row = {'Date': date}
            # 使用显示名称
            for asset, ret in returns.items():
                display_name = ASSET_NAME_MAP.get(asset, asset)
                row[display_name] = ret
            quarter_returns.append(row)
        self.quarterly_return_df = pd.DataFrame(quarter_returns)

In [10]:
# %% [markdown]
# ## 回测执行函数

# %%
def run_backtest():
    # 下载国际ETF数据
    intl_data = {}
    for ticker in INTL_ETFS:
        data = download_intl_etf(ticker, START_DATE, END_DATE)
        if data is not None:
            intl_data[ticker] = data
    
    # 下载中国ETF数据
    china_data = {}
    for ticker in CHINA_ETFS:
        # 对于159892，调整开始日期
        if ticker == '159892':
            adjusted_start_date = '2021-10-01'  # 早于实际开始日期确保有足够数据
        else:
            adjusted_start_date = START_DATE
            
        data = download_china_etf(ticker, adjusted_start_date, END_DATE)
        if data is not None:
            china_data[ticker] = data
    
    # 合并所有数据
    data_dict = {**intl_data, **china_data}
    
    if not data_dict:
        print("\n错误: 没有成功下载数据")
        return
    
    print("\n=== 初始化回测引擎 ===")
    cerebro = bt.Cerebro()
    cerebro.broker.set_cash(INITIAL_CASH)
    cerebro.broker.setcommission(commission=COMMISSION)
    
    print("\n=== 添加数据 ===")
    for ticker, data in data_dict.items():
        try:
            # 确保列名正确
            data.columns = ['open', 'high', 'low', 'close', 'volume']
            
            # 检查数据长度
            if len(data) < 100:
                print(f"{ticker} 数据长度不足 ({len(data)}), 跳过")
                continue
                
            data_feed = bt.feeds.PandasData(
                dataname=data,
                fromdate=datetime.strptime(START_DATE, '%Y-%m-%d'),
                todate=datetime.strptime(END_DATE, '%Y-%m-%d'),
                datetime=None,
                open=0,
                high=1,
                low=2,
                close=3,
                volume=4,
                openinterest=-1
            )
            cerebro.adddata(data_feed, name=ticker)
            print(f"成功添加 {ticker} 数据 ({len(data)} 条记录)")
        except Exception as e:
            print(f"添加 {ticker} 数据失败: {str(e)}")
    
    if not cerebro.datas:
        print("\n错误: 没有有效数据")
        return None, None, None
    
    print("\n=== 添加策略和分析器 ===")
    cerebro.addstrategy(QuarterlyContrarianRotationWithDip)
    
    # 添加分析器
    cerebro.addanalyzer(btanalyzers.SharpeRatio, _name='sharpe', riskfreerate=0.0, annualize=True)
    cerebro.addanalyzer(btanalyzers.DrawDown, _name='drawdown')
    cerebro.addanalyzer(btanalyzers.TimeReturn, _name='time_return', timeframe=bt.TimeFrame.Years)
    cerebro.addanalyzer(btanalyzers.AnnualReturn, _name='annual_return')
    cerebro.addanalyzer(btanalyzers.TradeAnalyzer, _name='trade_analysis')
    
    print('\n=== 开始回测 ===')
    print(f'初始资金: {cerebro.broker.getvalue():.2f}')
    
    try:
        results = cerebro.run()
    except Exception as e:
        print(f"\n回测错误: {str(e)}")
        return None, None, None  # 返回三个None值
    
    if not results:
        print("\n回测未产生结果")
        return None, None, None  # 返回三个None值
    
    strat = results[0]
    print(f'\n初始资金：{INITIAL_CASH:.2f}')
    print(f'最终资金: {cerebro.broker.getvalue():.2f}')
    
    return cerebro, strat, data_dict

In [11]:
def analyze_results(cerebro, strat, data_dict):
    """分析回测结果并生成报告"""
    # 计算总交易日数
    total_trading_days = len(cerebro.datas[0])  # 使用第一个数据源的长度作为总交易日数
    
    # 计算年化收益率（基于交易日）
    trading_days_per_year = 252  # 通常一年有252个交易日
    years = total_trading_days / trading_days_per_year
    total_return = (cerebro.broker.getvalue() / INITIAL_CASH) - 1
    annualized_return = (1 + total_return) ** (1 / years) - 1 if years > 0 else 0
    
    # 计算Alpha (使用SPY作为基准) - 使用原始下载数据
    if 'SPY' in data_dict:
        spy_start = data_dict['SPY'].iloc[0]['close']
        spy_end = data_dict['SPY'].iloc[-1]['close']
        spy_return = (spy_end - spy_start) / spy_start if spy_start > 0 else 0
        spy_annualized = (1 + spy_return) ** (1 / years) - 1 if years > 0 else 0
        alpha = annualized_return - spy_annualized
    else:
        alpha = 0
    
    # 获取分析器结果
    sharpe_ratio = strat.analyzers.sharpe.get_analysis().get('sharperatio', 0)
    drawdown = strat.analyzers.drawdown.get_analysis()
    max_drawdown = drawdown.get('max', {}).get('drawdown', 0) / 100
    
    print("\n=== 回测结果 ===")
    print(f"总交易日数: {total_trading_days}")
    print(f"总收益率: {total_return * 100:.2f}%")
    print(f"年化收益率: {annualized_return * 100:.2f}%")
    print(f"夏普比率: {sharpe_ratio:.2f}")
    print(f"最大回撤: {max_drawdown * 100:.2f}%")
    print(f"Alpha: {alpha * 100:.2f}%")
    
    # 打印交易分析 - 修复数据结构问题
    trade_analysis = strat.analyzers.trade_analysis.get_analysis()
    
    # 调试输出 - 查看trade_analysis结构
    # print("\n交易分析原始数据结构:")
    # print(trade_analysis)
    
    # 正确提取交易统计数据
    if isinstance(trade_analysis.get('total'), dict):
        # 处理 {'total': {'total': 36}} 格式
        total_trades = trade_analysis['total'].get('total', 0)
    elif 'total' in trade_analysis:
        # 处理 {'total': 36} 格式
        total_trades = trade_analysis['total']
    else:
        total_trades = 0
    
    # 处理盈利交易次数
    if 'won' in trade_analysis:
        if isinstance(trade_analysis['won'], dict):
            won_trades = trade_analysis['won'].get('total', 0)
        else:
            won_trades = trade_analysis['won']
    else:
        won_trades = 0
    
    # 处理亏损交易次数
    if 'lost' in trade_analysis:
        if isinstance(trade_analysis['lost'], dict):
            lost_trades = trade_analysis['lost'].get('total', 0)
        else:
            lost_trades = trade_analysis['lost']
    else:
        lost_trades = 0
    
    # 手动计算胜率
    if total_trades > 0:
        win_rate = won_trades / total_trades * 100
    else:
        win_rate = 0
    
    print("\n=== 交易统计 ===")
    print(f"总交易次数: {total_trades}")
    print(f"盈利交易次数: {won_trades}")
    print(f"亏损交易次数: {lost_trades}")
    print(f"胜率: {win_rate:.2f}%")
        
    # 打印其他交易统计数据
    if 'long' in trade_analysis:
        print(f"多头交易次数: {trade_analysis['long'].get('total', 0)}")
    if 'short' in trade_analysis:
        print(f"空头交易次数: {trade_analysis['short'].get('total', 0)}")
    # if 'pnl' in trade_analysis and 'gross' in trade_analysis['pnl']:
    #     print(f"总盈亏: {trade_analysis['pnl']['gross']['total']:.2f}")
    #     print(f"平均盈利: {trade_analysis['pnl']['gross']['average']:.2f}")
    
    print("\n=== 交易明细 ===")
    if hasattr(strat, 'trade_df') and not strat.trade_df.empty:
        print(strat.trade_df.tail(10))
        strat.trade_df.to_csv('trade_details.csv', index=False)
        print("交易明细已保存到 trade_details.csv")
    else:
        print("无交易记录")
    
    print("\n=== 年度回报 ===")
    if hasattr(strat, 'annual_return_df') and not strat.annual_return_df.empty:
        print(strat.annual_return_df)
        strat.annual_return_df.to_csv('annual_returns.csv', index=False)
        print("年度回报已保存到 annual_returns.csv")
    else:
        print("无年度回报数据")
    
    print("\n=== 季度资产回报 ===")
    if hasattr(strat, 'quarterly_return_df') and not strat.quarterly_return_df.empty:
        print(strat.quarterly_return_df.tail())
        strat.quarterly_return_df.to_csv('quarterly_returns.csv', index=False)
        print("季度回报已保存到 quarterly_returns.csv")
    else:
        print("无季度回报数据")
    
    # 生成图表
    print("\n生成回测图表...")
    try:
        figs = cerebro.plot(style='line', iplot=False, volume=False, silent=True)
        if figs:
            for i, fig in enumerate(figs):
                fig.savefig(f'backtest_chart_{i}.png')
                print(f"回测图表已保存为 backtest_chart_{i}.png")
    except Exception as e:
        print(f"绘图失败: {str(e)}")
        print("尝试使用静态图表...")
        try:
            import matplotlib
            matplotlib.use('Agg')
            figs = cerebro.plot(style='line', iplot=False, volume=False, silent=True)
            if figs:
                for i, fig in enumerate(figs):
                    fig.savefig(f'backtest_chart_{i}.png')
                    print(f"回测图表已保存为 backtest_chart_{i}.png")
        except Exception as e_inner:
            print(f"静态图表也失败: {str(e_inner)}")

    # 返回结果字典
    results = {
        'total_trading_days': total_trading_days,
        'total_return': total_return,
        'annualized_return': annualized_return,
        'sharpe_ratio': sharpe_ratio,
        'max_drawdown': max_drawdown,
        'alpha': alpha,
        'total_trades': total_trades,
        'won_trades': won_trades,
        'lost_trades': lost_trades,
        'win_rate': win_rate,
        'trade_df': getattr(strat, 'trade_df', None),
        'annual_return_df': getattr(strat, 'annual_return_df', None),
        'quarterly_return_df': getattr(strat, 'quarterly_return_df', None)
    }
    
    return results

In [12]:
# %% [markdown]
# ## 主执行函数

# %%
def main():
    # 运行回测
    cerebro, strat, data_dict = run_backtest()

    if cerebro is None or strat is None:
        print("回测失败，无法继续分析")
        return None
    
    # 分析结果
    results = analyze_results(cerebro, strat, data_dict)
    
    return results

# %% [markdown]
# ## 执行回测
# 运行下面的单元格开始回测

# %%
if __name__ == '__main__':
    backtest_results = main()

正在下载国际ETF ETH-USD 数据 (2021-07-01 至 2025-08-10)...
ETH-USD 数据下载成功，1501 条记录，时间范围: 2021-07-01 至 2025-08-09
正在下载国际ETF QQQ 数据 (2021-07-01 至 2025-08-10)...
QQQ 数据下载成功，1031 条记录，时间范围: 2021-07-01 至 2025-08-08
正在下载国际ETF SPY 数据 (2021-07-01 至 2025-08-10)...
SPY 数据下载成功，1031 条记录，时间范围: 2021-07-01 至 2025-08-08
正在下载中国ETF 159980 数据 (2021-07-01 至 2025-08-10)...
159980 数据下载成功，997 条记录，时间范围: 2021-07-01 至 2025-08-08
正在下载中国ETF 513010 数据 (2021-07-01 至 2025-08-10)...
513010 数据下载成功，997 条记录，时间范围: 2021-07-01 至 2025-08-08
正在下载中国ETF 159892 数据 (2021-10-01 至 2025-08-10)...
159892 数据下载成功，926 条记录，时间范围: 2021-10-19 至 2025-08-08
正在下载中国ETF 159934 数据 (2021-07-01 至 2025-08-10)...
159934 数据下载成功，997 条记录，时间范围: 2021-07-01 至 2025-08-08
正在下载中国ETF 159985 数据 (2021-07-01 至 2025-08-10)...
159985 数据下载成功，997 条记录，时间范围: 2021-07-01 至 2025-08-08
正在下载中国ETF 510880 数据 (2021-07-01 至 2025-08-10)...
510880 数据下载成功，997 条记录，时间范围: 2021-07-01 至 2025-08-08
正在下载中国ETF 516780 数据 (2021-07-01 至 2025-08-10)...
516780 数据下载成功，997 条记录，时间范围: 2021-07-01 至 2025-08-